In [1]:
import os
import re
import cv2
import glob
import json
import time
import random
import string
import imageio
import collections
import numpy as np
from PIL import Image
from typing import Tuple
from shapely.geometry import Polygon
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import matplotlib.pyplot as plt

# Import dataset that relies on synthetic_anno.json for change detection annotations
from syntheticpairs_dataset import SyntheticPairsDataSet
from models.changedetection_model import ChangeDetectionModel, ChangeDetectionModelResNet101
from create_dataset_json import get_scene_name_from_file, find_files
from constants import Action

In [2]:
class_mapping = {
    Action.NULL.value : "NULL",
    Action.REMOVED.value : "REMOVED",
    Action.ADDED.value : "ADDED",
    Action.SHIFTED.value : "SHIFTED",
}

latest_model_resnet50 = "change_detection_model_20240518221803.pt"
latest_model_resnet101 = "change_detection_model_resnet101_20240519040834.pt"

In [3]:
class_mapping 

{0: 'NULL', 1: 'REMOVED', 2: 'ADDED', 3: 'SHIFTED'}

In [4]:
# This dictionary defines hyperparameters for the dataset
# Complete configurations for each experiment can be found in the configs folder
test_set_params = {
        "dataset_name": "syntheticpairs",
        "root": "D:\\DeepLearningFiles\\renders_multicam_diff_all",
        "loader_params": {
            "batch_size": 8,
            "drop_last": False,
            "pin_memory": True,
            "num_workers": 8
        },
        "mode": "test",
        "crop": True,
        "resize": True,
        "spatial_resolution": [256, 180],
        "overfit": False,
        "normalize": True,
        "augment": False,
        "shuffle": False
    }

model_params = dict({
        "model_name": "changedetection",
        "num_classes": 4,
        "dataset_name": "syntheticpairs",
        "rgb": True, # Use RGB inputs
        "depth": False, # Use depth inputs
        "spatial_resolution": [256, 180],
        "mode": "test",
        "max_epochs": 3, # originally set to 4000
        "lr": 0.0004,
        "save_path": "/app/saved_models/changedetection_benchmark/",
        "weights_path": "/app/saved_models/changedetection_benchmark/",
        "load_weights": -1,
        "val_epochs": [1950],
        "val_rate": 50,
        "save_rate": 50,
        "lr_policy": "cosine",
        "lr_decay_iters": 10,
        "batch_size": 16,
        "loss_weights": True
    })

In [5]:
def calculate_iou(predictions, labels, num_classes):
    iou_per_class = []
    
    # Flatten lists
    predictions = [item for sublist in predictions for item in sublist]
    labels = [item for sublist in labels for item in sublist]
    
    predictions = np.array(predictions)
    labels = np.array(labels)
    
    for cls in range(num_classes):
        pred_cls = (predictions == cls)
        label_cls = (labels == cls)
        
        intersection = np.logical_and(pred_cls, label_cls).sum()
        union = np.logical_or(pred_cls, label_cls).sum()
        
        if union == 0:
            iou_per_class.append(np.nan)  
        else:
            iou_per_class.append(intersection / union)
    
    return iou_per_class

In [6]:
testset = SyntheticPairsDataSet(test_set_params)
testloader = torch.utils.data.DataLoader(testset, batch_size=test_set_params["loader_params"]["batch_size"],
                                          shuffle=test_set_params["shuffle"], num_workers=test_set_params["loader_params"]["num_workers"])
model = ChangeDetectionModel(model_params).model

if torch.cuda.is_available():
    model = model.cuda()
    model = torch.nn.DataParallel(model)

model.load_state_dict(torch.load(latest_model_resnet50))

c:\Users\Ugne\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Ugne\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Loaded weights into backbone
Number total params  39643962


<All keys matched successfully>

In [7]:
model_resnet101 = ChangeDetectionModelResNet101(model_params).model

if torch.cuda.is_available():
    model_resnet101 = model_resnet101.cuda()
    model_resnet101 = torch.nn.DataParallel(model_resnet101)

model_resnet101.load_state_dict(torch.load(latest_model_resnet101))

Loaded weights into backbone
Number total params  39643962
Loaded weights into backbone
Number total params  58636090


<All keys matched successfully>

In [8]:
def test_model(model, class_mapping, num_classes):
    model.eval()
    model_inputs = []
    model_labels = []
    model_outputs = []

    with torch.no_grad():
        for i, data in enumerate(testloader):
                # Process data to correct types
                img1, img2, label, scene, depth1, depth2 = data
                img1 = img1.float()
                img2 = img2.float()
                depth1 = depth1.float()
                depth2 = depth2.float()
                label = label.type(torch.LongTensor)
                
                # Move data to GPU if available
                if torch.cuda.is_available():
                    img1 = img1.cuda()
                    img2 = img2.cuda()
                    depth1 = depth1.cuda()
                    depth2 = depth2.cuda()
                    label = label.cuda()
                
                # Get model output
                model_input = (img1, img2, depth1, depth2)
                model_out = model(model_input)
                _, predicted_labels = torch.max(model_out.data, 1)

                model_inputs.extend(img1.cpu().numpy())
                model_outputs.append(predicted_labels.cpu().numpy().tolist())
                model_labels.append(label.cpu().numpy().tolist())

        ious = calculate_iou(model_outputs, model_labels, num_classes)

        for idx, iou in enumerate(ious):
            print(f"Class {class_mapping[idx]} IoU: {iou:.4f}")

        return model_inputs, model_outputs, model_labels, ious

## Results

### DeepLabV3 with Resnet50 backbone

In [9]:
model_resnet50_results = test_model(model, class_mapping, 4)

Class NULL IoU: 0.7661
Class REMOVED IoU: nan
Class ADDED IoU: 0.0257
Class SHIFTED IoU: 0.0242


### DeepLabV3 with Resnet101 backbone

In [10]:
model_resnet101_results = test_model(model_resnet101, class_mapping, 4)

Class NULL IoU: 0.7333
Class REMOVED IoU: nan
Class ADDED IoU: 0.0277
Class SHIFTED IoU: 0.0004
